### Ejercicio 1

A partir de la API de Pokemon, se pide generar un DataFrame como el de la imagen con las columnas: 

- **"height","id","order","weight","types"**

### Ejercicio 2

¿Has visto que dentro del DataFrame la columna "types" hay diccionarios? Se pide agregar, por cada pokemon, las columnas necesarias al DataFrame para albergar toda la información de "types" (solo las keys)

![](df_pk.PNG)

## EJERCICIO 1

In [22]:
import json
import requests

import pandas as pd
from pandas.io.json import json_normalize

url = "https://pokeapi.co/api/v2/pokemon?offset=0&limit=5"
poke_json = requests.get(url).json()
df = pd.DataFrame(poke_json['results'])
poke_json['results']

[{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
 {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
 {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
 {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
 {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'}]

In [26]:
# añadir los campos a los pokemons
for pokemon in poke_json['results']:
    p = requests.get(pokemon['url']).json()
    pokemon['height'] = p['height']
    pokemon['id'] = p['id']
    pokemon['order'] = p['order']
    pokemon['weight'] = p['weight']
    pokemon['types'] = p['types']

df_pokemons = pd.DataFrame(poke_json['results'])
df_pokemons

,name,url,height,id,order,weight,types
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
3,charmander,https://pokeapi.co/api/v2/pokemon/4/,6,4,5,85,"[{'slot': 1, 'type': {'name': 'fire', 'url': '..."
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,11,5,6,190,"[{'slot': 1, 'type': {'name': 'fire', 'url': '..."


## EJERCICIO 2


In [76]:
# Pruebas para ver como 'navegar' en el diccionario del pokemon
df_pokemons_types = df_pokemons.copy()
df_pokemons_types.drop(columns=['url', 'height', 'id', 'order', 'weight'],inplace=True)
df_pokemons_types['types'][0][0]['type']['name']
df_pokemons_types['types'][0]

[{'slot': 1,
  'type': {'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'}},
 {'slot': 2,
  'type': {'name': 'poison', 'url': 'https://pokeapi.co/api/v2/type/4/'}}]

In [118]:
df_pokemons_types2 = df_pokemons.copy()
df_pokemons_types2['type 1'] = df_pokemons_types2['types'].apply(lambda x: x[0]['type']['name'])

df_pokemons_types2['type 2'] = df_pokemons_types2['types'].apply(lambda x: x[1]['type']['name'] if len(x) == 2 else '')

df_pokemons_types2['type 3'] = df_pokemons_types2['types'].apply(lambda x: x[2]['type']['name'] if len(df_pokemons_types2['types'][0]) == 3 else '')
df_pokemons_types2.drop(columns=['types'], inplace=True)
df_pokemons_types2
# salida de los tipos de cada pokemon por columnas (fallo de lectura del enunciado)

,name,url,height,id,order,weight,type 1,type 2,type 3
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,grass,poison,
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,grass,poison,
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,grass,poison,
3,charmander,https://pokeapi.co/api/v2/pokemon/4/,6,4,5,85,fire,,
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,11,5,6,190,fire,,


In [97]:
df_pokemons_types3 = df_pokemons.copy()
df_pokemons_types3['types'] = df_pokemons_types3['types'].apply(lambda x: [y['type']['name'] for y in x])
df_pokemons_types3
# ídem

,name,url,height,id,order,weight,types
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,"[grass, poison]"
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,"[grass, poison]"
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,"[grass, poison]"
3,charmander,https://pokeapi.co/api/v2/pokemon/4/,6,4,5,85,[fire]
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,11,5,6,190,[fire]


In [115]:
# unión de las tablas 3 (con los datos de los pokemons) y 4 (slots 'explotados') 
df_pokemons_types4 = df_pokemons.copy()
df_pokemons_types4 = pd.DataFrame(df_pokemons_types4['types'].explode())

df_pokemons_types4['index'] = df_pokemons_types4.index
df_pokemons_types3['index'] = df_pokemons_types3.index

df_all = df_pokemons_types3.merge(df_pokemons_types4, on = 'index')
df_all.drop(columns= ['types_x','index'], inplace=True)
df_all

,name,url,height,id,order,weight,types_y
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,"{'slot': 1, 'type': {'name': 'grass', 'url': '..."
1,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,"{'slot': 2, 'type': {'name': 'poison', 'url': ..."
2,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,"{'slot': 1, 'type': {'name': 'grass', 'url': '..."
3,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,"{'slot': 2, 'type': {'name': 'poison', 'url': ..."
4,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,"{'slot': 1, 'type': {'name': 'grass', 'url': '..."
5,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,"{'slot': 2, 'type': {'name': 'poison', 'url': ..."
6,charmander,https://pokeapi.co/api/v2/pokemon/4/,6,4,5,85,"{'slot': 1, 'type': {'name': 'fire', 'url': 'h..."
7,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,11,5,6,190,"{'slot': 1, 'type': {'name': 'fire', 'url': 'h..."


In [121]:
# Solución más limpia
df_pokemons_all2 = df_pokemons.copy().explode('types')
df_pokemons_all2.reset_index(inplace=True)
df_pokemons_all2.drop(columns=['index'], inplace=True)
df_pokemons_all2
# A partir se extraen los valores de la columna 'types' y se generan nuevas columnas

,name,url,height,id,order,weight,types
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,"{'slot': 1, 'type': {'name': 'grass', 'url': '..."
1,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,"{'slot': 2, 'type': {'name': 'poison', 'url': ..."
2,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,"{'slot': 1, 'type': {'name': 'grass', 'url': '..."
3,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,"{'slot': 2, 'type': {'name': 'poison', 'url': ..."
4,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,"{'slot': 1, 'type': {'name': 'grass', 'url': '..."
5,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,"{'slot': 2, 'type': {'name': 'poison', 'url': ..."
6,charmander,https://pokeapi.co/api/v2/pokemon/4/,6,4,5,85,"{'slot': 1, 'type': {'name': 'fire', 'url': 'h..."
7,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,11,5,6,190,"{'slot': 1, 'type': {'name': 'fire', 'url': 'h..."


In [126]:
# hay que generar las columnas 'slot', 'type' y 'url_type'
df_pokemons_all2['Slot'] = df_pokemons_all2['types'].apply(lambda x: x['slot'])
df_pokemons_all2['type'] = df_pokemons_all2['types'].apply(lambda x: x['type']['name'])
df_pokemons_all2['url_type'] = df_pokemons_all2['types'].apply(lambda x: x['type']['url'])
df_pokemons_all2.drop(columns='types', inplace=True)

df_pokemons_all2

,name,url,height,id,order,weight,Slot,type,url_type
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,1,grass,https://pokeapi.co/api/v2/type/12/
1,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,2,poison,https://pokeapi.co/api/v2/type/4/
2,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,1,grass,https://pokeapi.co/api/v2/type/12/
3,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,2,poison,https://pokeapi.co/api/v2/type/4/
4,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,1,grass,https://pokeapi.co/api/v2/type/12/
5,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,2,poison,https://pokeapi.co/api/v2/type/4/
6,charmander,https://pokeapi.co/api/v2/pokemon/4/,6,4,5,85,1,fire,https://pokeapi.co/api/v2/type/10/
7,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,11,5,6,190,1,fire,https://pokeapi.co/api/v2/type/10/
